<a href="https://colab.research.google.com/github/simulate111/Climatic_Data/blob/main/MEPS_(MetCoOp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
pip install pandas

In [14]:
import pandas as pd
import requests
import io
import time

cities = {
    "Turku": {"lat": 60.45, "lon": 22.26},
    "Copenhagen": {"lat": 55.67, "lon": 12.56},
    "Stockholm": {"lat": 59.32, "lon": 18.06},
    "Oslo": {"lat": 59.91, "lon": 10.75}
}

print("Fetching 2024 MEPS Data with formatted time columns...")

for city, coords in cities.items():
    url = (
        f"https://historical-forecast-api.open-meteo.com/v1/forecast?"
        f"latitude={coords['lat']}&longitude={coords['lon']}&"
        f"start_date=2024-01-01&end_date=2024-12-31&"
        f"hourly=shortwave_radiation,temperature_2m,wind_speed_10m&"
        f"models=metno_nordic&"
        f"wind_speed_unit=ms&format=csv"
    )

    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"❌ Error {response.status_code} for {city}")
            continue

        # 1. Load data
        df = pd.read_csv(io.StringIO(response.text), skiprows=3)

        # 2. Convert 'time' to actual datetime objects
        df['time'] = pd.to_datetime(df['time'])

        # 3. Create the 'Hour' column in 01:00 format
        df['Hour'] = df['time'].dt.strftime('%H:%M')

        # 4. (Optional) Create a 'Date' column for easier filtering
        df['Date'] = df['time'].dt.strftime('%Y-%m-%d')

        # 5. Rename original variables for clarity
        df = df.rename(columns={
            'shortwave_radiation (W/m²)': 'GHI (W/m2)',
            'temperature_2m (°C)': 'Air Temp (C)',
            'wind_speed_10m (m/s)': 'Wind Speed (m/s)'
        })

        # Reorder columns to put Date and Hour first
        cols = ['Date', 'Hour', 'GHI (W/m2)', 'Air Temp (C)', 'Wind Speed (m/s)']
        df = df[cols]

        filename = f"{city.lower()}_meps_formatted_2024.csv"
        df.to_csv(filename, index=False)
        print(f"✅ {city} saved to {filename}")

        time.sleep(1)

    except Exception as e:
        print(f"❌ Failed {city}: {e}")

print("\nFormatting complete.")

Fetching 2024 MEPS Data with formatted time columns...
✅ Turku saved to turku_meps_formatted_2024.csv
✅ Copenhagen saved to copenhagen_meps_formatted_2024.csv
✅ Stockholm saved to stockholm_meps_formatted_2024.csv
✅ Oslo saved to oslo_meps_formatted_2024.csv

Formatting complete.
